## MoSDeF-Signac Tutorial
### Solvated Surface Screening
---

### Tutorial summary
This tutorial aims to familiarize a molecular simulations researcher on the ways to 
control a workflow managed by [Signac.](https://docs.signac.io/en/latest/) This is
an important tool for accessing and storing workflow metadata such that the workflow
is standardized and [extensible](https://doi.org/10.1080/00268976.2020.1742938), as 
part of the TRUE nature of computational research.

The following workflows will show how to:
1. Calculate the interfacial structures of different water models. Water models include:
    - SPC
    - SPC/E
    - SPC-flex
    - TIP3P-original
    - TIP3P-modified
    - OPC 
2. Build functional workflows that pull specifications from Signac job criteria.
3. Run HOOMD simulations and process data in a standardized format for easy extension of the project aims.

### Learning Objectives
1. How to operate/set up a [Signac](https://signac.io/) workflow.
2. Using [mBuild](https://mbuild.mosdef.org/en/stable/) recipes and scripts for functional workflows.
3. How to pass job metadata to a simulation process.
4. [HOOMD-blue](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulations.
5. Data processing.

### Sections
0. Google Colab Setup and import packages
1. Initialize Signac Project
2. Generate mBuild Structures
3. Apply Force Fields
4. Run HOOMD-blue Simulations
5. Analyze/Record Results

## 0. Google Colab Setup
---
Run these next three panes only if you are using Google Collab for this notebook

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

import condacolab
condacolab.check()

!conda install mamba
!git clone https://github.com/mosdef-hub/reproducibility_study.git
!mamba env update -n base -f reproducibility_study/environment.yml

In [ ]:
!mamba install mbuild foyer hoomd gsd signac mdanalysis #gmso

In [ ]:
!git clone https://github.com/CalCraven/gmso.git
%cd gmso
!git fetch origin hoomd-testing-cecam
!pip install .
%cd ..
!git clone https://github.com/CalCraven/surface_coatings.git
%cd surface_coatings
!git fetch origin colab-installation
!git checkout colab-installation
!pip install .
%cd ..

In [ ]:
!git clone --branch solvated-surface-wkflw https://github.com/CalCraven/CECAM-MoSDeF-Workshop.git
%cd CECAM-MoSDeF-Workshop/solvated_surface_workflow/
!pwd

## 0. Import packages
---

In [ ]:
# Import Libraries
import itertools
import os

import signac
import mbuild as mb
import gmso
from gmso.parameterization import apply
import gsd
import unyt as u

from surface_coatings.monolayer import Monolayer
from surface_coatings.solvated_monolayer import SolvatedMonolayer
from surface_coatings.surfaces.silica_interface import SilicaInterface
from surface_coatings.surfaces.silica_interface_carve import SilicaInterfaceCarve
from surface_coatings.chains.alkylsilane import Alkylsilane

## 1. Initialize Signac Project
---
From the Signac Documentation:
"The signac framework supports researchers in managing project-related data with a well-defined indexable storage layout for data and metadata. This streamlines post-processing and analysis and makes data collectively accessible. The signac framework aims to help make computational research projects Transparent, Reproducible, Usable by others, and Extensible (TRUE) [TGM+20], a set of principles put forth by the MoSDeF Collaboration [CMI+21]."

#### Step 1.
We will use signac to define some project parameters. The parameters are stored in `jobs` that are a unique set of arguments that define some unique set of observations to make in the study. The following `grid` function is a standard way to generate all combinations of the specified statepoint, defined in the dictionary (`sp_gridDict`). 

Run the following cell and evaluate the project that is generated.

In [ ]:
def grid(gridspec):
    """Yields the Cartesian product of a `dict` of iterables.

    The input ``gridspec`` is a dictionary whose keys correspond to
    parameter names. Each key is associated with an iterable of the
    values that parameter could take on. The result is a sequence of
    dictionaries where each dictionary has one of the unique combinations
    of the parameter values.
    """
    for values in itertools.product(*gridspec.values()):
        yield dict(zip(gridspec.keys(), values))

project = signac.init_project("./solvated_surface_project")
# generate statepoints:
sp_gridDict = {
    "water_model":["spce"], # forcefield to use
    "temperature":[298.15],  # K
    "chain_density":[3], # chains/nm 
    "chain_length":[4], # n_carbons
    "solvent_box_height": [3], #nm
    "seed":[42901423],
}
        
for sp in grid(sp_gridDict):
    print("Initializing job", sp)
    project.open_job(sp).init()
project

The project creation generates a directory on the hard drive with information related to the project defintion. Here are a list of some of the files.

In [ ]:
!ls -a solvated_surface_project/

The workspace is a directory where the individual jobs that were initialized above are stored. These are differentiated by unique hashes generatedfrom each statepoint, and so there's no risk of accidentally overwriting data for a different job.

Let's look at the `signac_statepoint.json` file in one of our jobs.

In [ ]:
!cat solvated_surface_project/workspace/f959300be61c911cc8512fd1619077d2/signac_statepoint.json

Nice work! You should see something that looks like a python dictionary with all of the information that was passed in the above cells. For more information on how to access this job metadata, see the following [signac tutorials.](https://docs.signac.io/en/latest/tutorial.html#interacting-with-the-signac-project)

### Exercise 1: Try to add a new job to the workspace. Validate that the directory is added properly

In [ ]:
project = signac.get_project("./solvated_surface_project/")
jobSP = { #initialize a dictionary statepoint
    "water_model":"spce", # forcefield to use
    "temperature":???,  # K
    "chain_density":???, # chains/nm 
    "chain_length":???, # n_carbons
    "solvent_box_height": 3, #nm
    "seed":42901423,
}
project.??? # open and initialize the jobSP dictionary you made
project # how many jobs are there now?

## 2. Generate mBuild Structures
---

#### Step 2.
Next we will use `mBuild` (which you're already an expert in) to build up the individual components of our surface. The key components in this hierarchical structure are:
1. The surface, made from silica. This will be loaded via premade recipe in the [coated_surfaces](https://github.com/daico007/surface_coatings/tree/main) repository.
2. The chains to attach to the surface. These are alkylsilane chains, built using the [Polymer Builder](https://github.com/mosdef-hub/mbuild/blob/main/mbuild/lib/recipes/polymer.py) recipe in `mBuild`.
3. The water atoms, which are simple three site HOH molecules, made from the [SMILES string](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html). 

In [ ]:
job_compoundDict = {} # dictionary to store job compounds

# load the water
water = mb.load("O", smiles=True)
water.name = "water"
solvent_density = 0.35 * u.g/u.cm**3 # calculate density in g/cm^3
atomic_density = solvent_density / (u.amu.to("g")) / water.mass # convert density to atoms/nm^3
print(f"Atoms/nm^3 is: {atomic_density.to('nm**-3'):.0f}")

for job in project: #iterate over all jobs using a simple for loop on the `project`
    chain = Alkylsilane(chain_length=job.sp.chain_length) #create a chain
    chain.energy_minimize()
    surface = SilicaInterfaceCarve(thickness=1.2)
    surface_area = surface.box.Lx * surface.box.Ly
    n_chains=int(job.sp.chain_density*surface_area)
    n_chains=5
    graft_pattern = mb.Random2DPattern(n_chains, seed=job.sp.seed)
    monolayer = Monolayer(surface, graft_pattern, chain, n_chains=n_chains)
    monolayer.name = "surface"
    n_waters = int(surface_area*job.sp.solvent_box_height*atomic_density.to("nm**-3").value)
    print(f"Adding {n_waters:d} waters to the system.")
    solvated_monolayer = SolvatedMonolayer(monolayer, water, n_waters, job.sp.solvent_box_height)
    job_compoundDict[job] = { # store the saved components for visualization
        "water":water, 
        "chain": chain, 
        "surface":surface, 
        "solvated_monolayer":solvated_monolayer
    }


Let's visualize the monolayer that was built.

In [ ]:
solvated_monolayer.visualize()

#### Exercise 2: The water was packed at too low of a density.
Try to pack the water at a higher density. Find the line
`solvent_density = 0.35 * u.g/u.cm**3 # g/cm^3` 
and replace `0.35` with `0.85` for a better packed system. Note that this may take longer due the packing algorithms.

Also, there looks like there's a gap in between the water box and the top of the surface. Fix that by adding


`solvated_monolayer.children[1].translate([0,0,-0.3])`


To move just the water molecules down by 0.3 nm. This should be added after the line


`solvated_monolayer = SolvatedMonolayer(monolayer, water, n_waters, job.sp.solvent_box_height)`

## 3. Apply Force Fields
---

#### Step 3. 
We must now use force fields that define the potential equations of this system. The force fields are stored on disk as XML files. These define all of the forces needed for the simulation. Multiple force fields can be applied to different molecules in the `mBuild` compound but be certain that these force fields are compatible (i.e. same mixing rule, 1-4 scaling, cutoffs). Also, while these force fields are stored in a generalized format, not all simulation engines can support all force field types, so keep that in mind when selecting a force field to use.

We can iterate through the jobs, grab the solvated surfaces, load the water force field specified in `job.sp.water_model` from the `xmls/` directory in this project, and create our parameterized GMSO `Topology`. 

First though, let's look at the syntax for the apply step. 

In [ ]:
from gmso.parameterization import apply
help(apply)

Now let's apply the [OPLSAA force field](https://pubs.acs.org/doi/10.1021/ja9621760) to the surface and the specified water force field from the `job.sp.water_model`.

In [ ]:
surfaceFF = gmso.ForceField("xmls/oplsaa.xml") # load a general opls all atom forcefield from the directory

for job in project:
    ff_path = os.path.join("xmls", job.sp.water_model+".xml")
    waterFF = gmso.ForceField(ff_path)
    top = job_compoundDict[job]["solvated_monolayer"].to_gmso()
    top.identify_connections() # generate bonds, angles, and dihedrals
    parameterized_top = apply(
        top, forcefields={"surface": surfaceFF, "water":waterFF}, 
    )
    job_compoundDict[job]["parameterized_top"] = parameterized_top
                              
parameterized_top   

#### Exercise 3a: Identify the aspectes of the parameterized topology
What is the bonded equation that is being used from both of our forcefields?

To identify this, we need to interrogate the `parameterized_topology` object. Components are stored in IndexedSets. These can be accessed via list indexing.

In [ ]:
parameterized_top.bonds[?]???_type.expression # access index 0 and print then access the bond_type.expression
# this can also be done with angles, dihedrals, and impropers
parameterized_top.angles.???

#### Answer should look something like this:
0.5𝑘(𝑟−𝑟𝑒𝑞)2

#### Exercise 3b: What is the number of unique bond types in the topology?

To identify that, use a filter to look for bond_types with a unique pair of atom classes

Potential filters allow the user to specify what is `unique` about the type. These give a flexible set of methods to get the unique types in the system. For instance, there may be a set with unique parameters. Maybe you want to look at the set with unique full potential expressions. In this case, we will sort by the unique names of the atom_classes that make up the bonded atoms.

In [ ]:
from gmso.core.views import PotentialFilters

pfilter = PotentialFilters.UNIQUE_NAME_CLASS
print(f"There are {len(parameterized_top.bond_types(filter_by=pfilter))} uniquely named bond types")
# try a different filter from the PotentialFilters class

## 4. Run HOOMD-blue Simulations
---

#### Step 4.
Now let's run our [HOOMD](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulations for each job. We create a generalized `run_hoomd` python function here which accepts whatever we write out from the GMSO Topology `parameterized_top`.

In [ ]:
from gmso.external import to_hoomd_snapshot, to_hoomd_forcefield
import hoomd
import numpy as np
n_steps=5000
"""Simulation configuration and runtime parameters."""
base_units = { # this unit sytems lets hoomd non-dimensionalize all parameters in the forcefield
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}
job = project.open_job(id="fdb859acfdae44e289ad701c747452f1")
parameterized_top = job_compoundDict[job]["parameterized_top"]
gmso_snapshot, snapshot_base_units = to_hoomd_snapshot( # TODO: simulation forces and connections are ordered differently
    parameterized_top, base_units=base_units
)
gmso_forces, forces_base_units = to_hoomd_forcefield( #TODO: can't handle dimensionless parameters currently, PR incoming
    parameterized_top,
    r_cut=0.6,
    base_units=base_units,
    pppm_kwargs={"resolution": (32,32,32), "order": 7},
)


filter_ = hoomd.filter.Tags(list(np.arange(1500, parameterized_top.n_sites+1)))
#filter_ = hoomd.filter.Tags(list(np.arange(2100, top.n_sites+1)))
#filter_ = ZSliceFilter(min_position+0.1)


cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

sim.create_state_from_snapshot(gmso_snapshot)
temp = job.sp.temperature * u.K
kT = temp.to_equivalent("kJ/mol", "thermal").value
sim.state.thermalize_particle_momenta(filter=filter_, kT=kT)

fire = hoomd.md.minimize.FIRE(
    dt=0.001,
    force_tol=1e-1,
    angmom_tol=1e-1,
    energy_tol=1e-5,
    forces=list(set().union(*gmso_forces.values()))
)
displacement_capped = hoomd.md.methods.DisplacementCapped(
    filter=filter_,
    maximum_displacement=1e-3)
fire.methods.append(displacement_capped)
sim.operations.integrator = fire


outlogger = hoomd.logging.Logger(categories=['scalar', 'string'])
outlogger.add(sim, quantities=['timestep', 'tps'])
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=filter_
)
sim.operations.computes.append(thermodynamic_properties)
outlogger.add(thermodynamic_properties, ['kinetic_temperature', "potential_energy"])
table = hoomd.write.Table(
    trigger=hoomd.trigger.Periodic(period=100),
    logger=outlogger
)
sim.operations.writers.append(table)


while not(fire.converged):
    sim.run(1000)
    if sim.timestep > 50000:
        break

In [ ]:
displacement_capped = hoomd.md.methods.DisplacementCapped(
    filter=filter_,
    maximum_displacement=1e-1)
fire.methods = [displacement_capped]
while not(fire.converged):
    sim.run(1000)
    if sim.timestep > 50000:
        break

#sim.operations.writers.remove(table)
#sim.operations.integrator = []

#sim.operations.integrator = hoomd.md.Integrator(dt=0.0001)
#sim.operations.integrator.forces.extend(
#    list(set().union(*gmso_forces.values()))
#)
#thermostat = hoomd.md.methods.thermostats.Bussi(kT)
#nvt = hoomd.md.methods.ConstantVolume(
#    thermostat=thermostat, filter=filter_
#)
#sim.operations.integrator.methods.append(nvt)
#sim.state.thermalize_particle_momenta(filter=filter_, kT=kT)
#thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
#    filter=filter_
#)

## 5. Analyze/Record Results
---

#### Step 5. 
Plot the simulation energies.

In [ ]:
import gsd
job = project.open_job(id="fdb859acfdae44e289ad701c747452f1")
data_path = job.fn("trajectory.gsd")
data = gsd.hoomd.read_log(data_path)
timestep = data['configuration/step']
potential_energy = data[
    'log/md/compute/ThermodynamicQuantities/potential_energy']

fig, ax = plt.subplots(1,1,figsize=(5, 3.09))
ax.plot(timestep, potential_energy)
ax.set_xlabel('timestep')
ax.set_ylabel('potential energy')
fig.show()

#### Exercise 5.
Load the trajectories into MDAnalysis. Lots of typical simulation analysis is then accessible for the trajectory.

In [ ]:
from MDAnalysis import Universe
for job in project:
    universe = Universe(job.fn("trajectory.???")) # <-- use the gsd loader. 
    #From here, you can convert the trajectory into many different options, 
    # Such as mdtraj gromacs, and freud.